<a href="https://colab.research.google.com/github/MarioRMReis/Sentinel_Images/blob/main/EE_ImageRequest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import the required libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
import geemap
import os
import cv2
import requests
import math
from pykml import parser
%matplotlib inline

# Log-in into Earth Engine

In [2]:
import ee
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AWtgzh7WvGeuLUrr4tIHl2a-oxMCAQHPrWcife0C2JW2Rbxif3Xb5zv322o

Successfully saved authorization token.


# Parameters for the image request (C: drive path, Starting data, Ending date, Max number of images)

In [3]:
C_drivePath = 'C:/Users/mario/Documents/Mario/'
folder_dataset = C_drivePath + 'Sentinel/dataset'
starting_date = '2017-01-01'
ending_date = '2023-12-31'
nimgs = 2190

# Read KML files and extract all regions of interest 

In [5]:
aoi = []
aoi_names = []
kml_files = os.listdir(C_drivePath + 'Sentinel/roi/')

for kml in kml_files:
  with open(C_drivePath + 'Sentinel/roi/' + kml, 'r') as f:
    root = parser.parse(f).getroot()
  namespace = {"kml": 'http://www.opengis.net/kml/2.2'}
  pms = root.xpath(".//kml:Placemark[.//kml:Polygon]", namespaces=namespace)
  roi_string = []
  for p in pms:
    if hasattr(p, 'MultiGeometry'):
      for poly in p.MultiGeometry.Polygon:
        roi_string.append(poly.outerBoundaryIs.LinearRing.coordinates)
    else:
      roi_string.append(p.Polygon.outerBoundaryIs.LinearRing.coordinates)

  dot = kml_files[0].find('.')
  name_kml = kml_files[0][:dot] + '-'
  for jdx, r in enumerate(roi_string):
    aoi_names.append(name_kml + str(jdx))
    roi_str = str(r).split(' ')
    aux = []
    for idx, rs in enumerate(roi_str):
      if idx == 0:
        n = rs[7::].split(',')
        aux.append([float(n[i]) for i in range(len(n)-1)])
      elif idx == (len(roi_str)-1): 
        print(rs)
      else:
        n = rs.split(',')
        aux.append([float(n[i]) for i in range(len(n)-1)])
    aoi.append([aux])

# Get the fitted Area Of Interest for a 128x128 
> The aoi will be at the center of the 128x128 image



In [6]:
def get_squares(aois):
    # 0.011377373345324189 0.014894925631009616 -> Height, Width
    S128 = [0.014854925631009616, 0.011377373345324189]
    new_aois = []
    h_aux = S128[1]/2
    w_aux = S128[0]/2

    for aoi in aois:
      # Find centeroid ----------------------------
      siz = len(aoi[0])
      list_width = []
      list_height = []
      for i in range(siz): 
        list_width.append(aoi[0][i][0])
        list_height.append(aoi[0][i][1])

      max_width = max(list_width)
      min_width = min(list_width)
      max_height = max(list_height)
      min_height = min(list_height)
      
      centroid_width = (max_width+min_width)/2
      centroid_height = (max_height+min_height)/2

      aoi_new = [[[(centroid_width-w_aux), (centroid_height+h_aux)], [(centroid_width-w_aux), (centroid_height-h_aux)], [(centroid_width+w_aux), (centroid_height-h_aux)], [(centroid_width+w_aux), (centroid_height+h_aux)]]]
      #inside, split_aois = check_if_inside(aoi_new, aoi)
      #if inside:
      new_aois.append(aoi_new)
      size = [128,128]
      #else:
        #for saois in split_aois:
          #new_aois.append(saois)

      
      #ans = image_ratio(S128, [list_width, list_height])
    return new_aois, size

# For all AOIs Create the mask and save it in the mask folder for the aoi

In [15]:
def get_mask(path, aoi, size):
  # Inputs:
  #     - path -> path to the aoi saved images
  #     - aoi -> Area of interest of the landfield
  #     - size = [sizeX, sizeY] -> size of the mask, (ie: 128x128, 256x256, 354x128...)
  aoi_mask = ee.Geometry.Polygon(aoi,None,False)
  ffa_s2 = ee.ImageCollection('COPERNICUS/S2') \
                        .filterBounds(aoi_mask) \
                        .filterDate(ee.Date('2017-01-01'), ee.Date('2023-12-31'))
  colList = ffa_s2.toList(30)
  # This part get's the land area image needed to create the mask --------------
  img = ee.Image(colList.get(17)).double().clip(aoi_mask)
  rgb = ['B4','B3','B2']
  url = img.getThumbURL({"min":-200000, "max":-200000,"bands":rgb})

  img_data = requests.get(url).content
  try:
    os.makedirs(path + '/Mask')
    id = img.id().getInfo()
    with open(path + '/Mask/mask_aux.jpg', 'wb') as handler:
      handler.write(img_data)
  except:
    id = img.id().getInfo()
    with open(path + '/Mask/mask_aux.jpg', 'wb') as handler:
      handler.write(img_data)

  # load the saved image -------------------------------------------------------
  img_aux = cv2.imread(path + '/Mask/mask_aux.jpg')
  img_zeros = np.zeros([128, 128, 3])

  # Create mask ----------------------------------------------------------------
  # img_aux.shape -> [x,y,z]
  x = img_aux.shape[0]
  y = img_aux.shape[1]
  x_sum = math.ceil((size[0]-x)/2)
  y_sum = math.ceil((size[1]-y)/2)

  img_zeros = np.zeros([size[0], size[1], 3])
  img_zeros[x_sum:(x_sum+x), y_sum:(y_sum+y), :] = img_aux

  cv2.imwrite(path +'/Mask/mask.jpg', img_zeros)


In [16]:
aoi_square, size =  get_squares(aoi)
for idx, a in enumerate(aoi):
  path = folder_dataset + '/' + aoi_names[idx]
  get_mask(path, a, size)

# Extract Images from Sentinel-1

In [20]:
def ExportCol_Sentinel1(path, roi, channel, min, max, n):
  try:
    for i in range(0,nimgs,1):
      aoi_geometry = ee.Geometry.Polygon(roi ,None,False)
      ffa_s = ee.ImageCollection('COPERNICUS/S1_GRD') \
                        .filterBounds(aoi_geometry) \
                        .filterDate(ee.Date(starting_date), ee.Date(ending_date))

      colList = ffa_s.toList(nimgs); 

      img = ee.Image(colList.get(i)).double().clip(aoi_geometry)
      # Exeption beacause to get a RGB channel with the Sentinel-1 we need to correspond each channel and compute the last
      if channel == 'RGB':
        imgR = img.select('VV')
        imgG = img.select('VH')
        imgB = img.select('VV').divide(img.select('VH'))
        url = ee.Image.rgb(imgR,imgG,imgB).getThumbURL({'min': [min[0], min[1], 0], 'max': [0, 0, 2]})
      else:
        url = img.select(channel).getThumbURL({'min': min[n], 'max': max[n]})

      id = img.id().getInfo()
      img_data = requests.get(url).content

      # Creates the folder but the flag need to be turned off beacause it will try to makedir after the creation
      try:
        os.makedirs(path)
        with open(path + '/' + id +'.jpg', 'wb') as handler:
          handler.write(img_data)
          #create_folders = False
      except:
        # This just saves the image in the correct folder with the name 'id' (var->'id')
        with open(path + '/' + id +'.jpg', 'wb') as handler:
          handler.write(img_data)
  except:
    return



> The request of the images from Sentinel-1 all aoi's 



In [21]:
aoi_bands = ee.Geometry.Polygon(aoi[0],None,False)

ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD') 
                       .filterBounds(aoi_bands) 
                       .filterDate(ee.Date('2019-01-01'), ee.Date('2019-12-31')) 
                       .first() 
                       .clip(aoi_bands))

# Variables needed to save images
bands_s1 = ffa_db.bandNames().getInfo()
bands_s1.remove('angle')
bands_s1.append('RGB')

# Defined values to be the best for these bands
min = [-14, -25]
max = [-7] * 2
#-------------------
for idx, a in enumerate(aoi_square):
  for jdx, b in enumerate(bands_s1):
    path = folder_dataset +'/'+ aoi_names[idx] +'/'+ 'Sentinel-1/' + b
    ExportCol_Sentinel1(path, a, b, min, max, jdx)

# Extract images from Sentinel-2

In [ ]:
def ExportCol_Sentinel2(path, roi, channel,  min, max, n):
  for i in range(0,nimgs,1):
    try:
      aoi_geometry = ee.Geometry.Polygon(roi ,None,False)
      ffa_s = ee.ImageCollection('COPERNICUS/S2') \
                        .filterBounds(aoi_geometry) \
                        .filterDate(ee.Date(starting_date), ee.Date(ending_date))

      colList = ffa_s.toList(ffa_s.size());                   

      img = ee.Image(colList.get(i)).double().clip(aoi_geometry)
      # Exeption beacause to get a RGB channel with the Sentinel-1 we need to correspond each channel and compute the last
      if channel == 'RGB':
        rgb = ['B4','B3','B2']
        url = img.getThumbURL({"min":min[n], "max":max[n],"bands":rgb})
      else:
        url = img.getThumbURL({"min":min[n], "max":max[n],"bands":channel})

      id = img.id().getInfo()
      img_data = requests.get(url).content

      # Creates the folder but the flag need to be turned off beacause it will try to makedir after the creation
      try:
        os.makedirs(path)
        with open(path + '/' + id +'.jpg', 'wb') as handler:
          handler.write(img_data)
      except:
        # This just saves the image in the correct folder with the name 'id' (var->'id')
        with open(path + '/' + id +'.jpg', 'wb') as handler:
          handler.write(img_data)
    except:
      return



> The request, cloud coverage < 20% 



In [ ]:
aoi_bands = ee.Geometry.Polygon(aoi[0],None,False)

ffa_s2 = ee.Image(ee.ImageCollection('COPERNICUS/S2') 
                       .filterBounds(aoi_bands) 
                       .filterDate(ee.Date('2017-01-01'), ee.Date('2023-12-31')) 
                       .first() 
                       .clip(aoi_bands))

bands_s2 = ffa_s2.bandNames().getInfo()
aux_bands_s2 =  bands_s2.copy();
for x in aux_bands_s2: 
  if x in  ['QA10','QA20','QA60','B10']: 
    bands_s2.remove(x) #bands_s2.remove(['QA10','QA20','QA60','B10'])
bands_s2.append('RGB')

# Defined best values for these bands
min = [0] * len(bands_s2)
max = [2700, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 2000, 3200, 2500, 3000]

for idx, a in enumerate(aoi_square):
  for jdx, b in enumerate(bands_s2):
    path = folder_dataset +'/'+ aoi_names[idx] +'/'+ 'Sentinel-2/' + b
    ExportCol_Sentinel2(path, a, b, min, max, jdx)